In [ ]:
import numpy as np

from scipy.stats import gaussian_kde

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from ase.io import read, write
from ase.neighborlist import neighbor_list

from collections import Counter

from quippy.descriptors import Descriptor

from matscipy.rings import ring_statistics

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = [ 'Helvetica', 'Arial', 'FreeSans', 'DejaVu Sans' ]

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.borderaxespad'] = 1.5

plt.rcParams['lines.linewidth'] = 3.0
plt.rcParams['axes.linewidth'] = 1.5

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.0
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1.0

plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 1.0
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1.0


In [ ]:
def S_at_q(gs, rs, rho, q):
    
    integrand = []
    
    for i in range(len(rs)):
        term = (gs[i]-1) * (rs[i]**2) * np.sin(q*rs[i])/(q*rs[i])
        integrand.append(term)
    
    integral = np.trapz(integrand, rs)
    
    return 1 + 4*np.pi*rho * integral


In [ ]:
r_min = 0.01
r_max = 14.0
n_r_steps = 1400

all_as = []
all_pdfs = []
all_lens = []

#files = [ '03_test_Ge/Ge_gap_SOAP_5000_5cutoff/run_216001_slow_from1500K/out_liquid_111_to_120.xyz',
#          '03_test_Ge/Ge_gap_SOAP_5000_5cutoff/run_216001_slow_from1500K/out_amo_491_to_500.xyz' ]

#files = [ '03_test_Ge/Ge_2bSOAP_5000_125_216_d155/run_10001_full_Joe/out_amo_331_to_340.xyz' ]

files = [ '03_test_Ge/Ge_2bSOAP_5000_125_216_d155/run_10001_full_Joe/out_highT_liquid_91_to_100.xyz' ]

for f in files:
    
    print('processing file: ', f)
    
    aa = read(f, index=':')
    all_lens.append(len(aa[-1]))
    
    this_pdfs = []
    
    for a in aa:
        all_as.append(a.cell[0][0])

        d = neighbor_list('d', a, r_max)
        h, bin_edges = np.histogram(d, bins=n_r_steps, range=(r_min, r_max))
        this_pdf = h/(4*np.pi/3*(bin_edges[1:]**3 - bin_edges[:-1]**3)) * a.get_volume() / (len(a)**2)
        this_pdfs.append(this_pdf)

    all_pdfs.append(np.average(this_pdfs, axis=0))


In [ ]:
ds = np.linspace(r_min, r_max, n_r_steps)

plt.plot(ds, all_pdfs[0])
#plt.plot(ds, all_pdfs[1])

plt.show()


In [ ]:
ds = np.linspace(r_min, r_max, n_r_steps)
qs = np.linspace(0.01, 20, 2000)

all_sqs = []
all_qs = []

for ip in range(len(all_pdfs)):

    this_sqs = []

    for q in qs:
        this_sqs.append(S_at_q(all_pdfs[ip], ds, 1000 / (all_as[ip]**3), q))
    
    all_sqs.append(this_sqs)
    all_qs.append(qs)
    
# read experimental data for comparison
# TBD

qsExpt_liq, SqsExpt_liq = np.loadtxt('digitise_Sq/liquid_neutron_Salmon_1988_sorted.txt', unpack=True)
qsExpt_amo, SqsExpt_amo = np.loadtxt('digitise_Sq/aGe_neutron_Etherington_1982_sorted.txt', unpack=True)


In [ ]:
plt.plot(qsExpt_liq, SqsExpt_liq, 'o', c='silver', label='l-Ge, 1273 K (expt.)')
#plt.plot(qsExpt_amo, SqsExpt_amo/np.average(SqsExpt_amo[-20:]), 'o', c='silver', label='a-Ge (expt.)')

plt.plot(all_qs[0], all_sqs[0], '-', c='indigo', label='GAP')

plt.xlim(1,14)
plt.ylim(-0.25,2.5)

plt.ylabel('S(q)')
plt.xlabel('q (1/A)')
plt.legend(loc='best')

plt.tight_layout(pad=1.0)
#plt.savefig('plot_Sq.png', dpi=450)


In [ ]:
fig, axs = plt.subplots(2, figsize=(6.85,7))

axs[0].plot(qsExpt_liq, SqsExpt_liq, 'o', c='silver', label='l-Ge (expt. 1,273 K)')
#axs[0].plot(all_qs[0], all_sqs[0], c='indigo', label='ht liquid sim. (2,500 K)')

axs[0].set_xlim(1,14)
axs[0].set_ylabel('S(q)')
axs[0].legend(loc='best')

axs[1].plot(qsExpt_amo, SqsExpt_amo/np.average(SqsExpt_amo[-20:]), 'o', c='silver', label='a-Ge (expt.)')
#axs[1].plot(all_qs[1], all_sqs[1], c='indigo', label='amo_216')

axs[1].set_xlim(1,14)
axs[1].set_ylabel('S(q)')
axs[1].set_xlabel('q (1/A)')
axs[1].legend(loc='best')

#plt.xlim(0.5, 15)
#plt.ylim(-0.25, 2.5)
#plt.legend(loc='best')
